## This is the code I used to create the TP & FP datasets

It needs a little cleaning and simplifying, but this is the code I used to create:
* https://www.kaggle.com/tpmeli/true-positive-centered-10-second-spectrograms
* https://www.kaggle.com/tpmeli/false-positive-centered-10-second-spectrograms

This was done on my local machine so some code is specific to that.
For example - I show the specs here, but you can use "matplotlib.use('Agg')" to disable showing
the outputs for faster processing.


In [ ]:
from tqdm import tqdm
from glob import glob

import matplotlib.pyplot as plt
import matplotlib
# matplotlib.use('Agg')  # Doesn't display plots - Use for on your own computer

# Import and convert
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")

import librosa
import librosa.display
import librosa as lr
import os

from pathlib import Path

import pandas as pd

os.getcwd()

In [ ]:

# PUT YOUR OWN PATH HERE.
path = "../input/rfcx-species-audio-detection/"
train_tp = pd.read_csv(path + "train_tp.csv")

In [ ]:
print(train_tp.shape)
train_tp.head()

In [ ]:
len(train_tp)

## Read in Files ##

In [ ]:
base_dir = "../input/rfcx-species-audio-detection/train/"

# Glob creates a file path list of all of a certain filetype, in this case .flac.
flac_file_paths = list(glob(base_dir + "*.flac"))

print("Sample file paths:")
flac_file_paths[:3]

In [ ]:
train_tp["song_duration"] = train_tp["t_max"] - train_tp["t_min"]
train_tp["freq_range"] = train_tp["f_max"] - train_tp["f_min"]
train_tp.head()

In [ ]:
train_tp["song_duration"].hist()
plt.show()

Sond duration doesn't go about

In [ ]:
train_tp["freq_range"].hist()
plt.show()

In [ ]:
train_tp["freq_range"].min()

In [ ]:
train_tp["freq_range"].max()

### Create Spectrograms

I used this code on my local machine, so some code is specific to that.

Here I just do 10 specs, but you can do all of them easily by looping over train_tp (using all_iters)

In [ ]:
i = 0
show_specs = False
save_dir = ""

skipped = []

all_iters = len(train_tp)
just_ten = 10

process_these = just_ten

print(f"processing {process_these} files")

for i in tqdm(range(process_these)):
    
    try:
        this_row = train_tp.iloc[i]

        this_filename = base_dir + str(this_row.recording_id) + ".flac"

        print("processing: ", this_row.recording_id)

        # Important information about centering the audio array.
        song_start = this_row["t_min"]
        song_end = this_row["t_max"]
        song_duration = song_end - song_start
        clip_length = 10
        empty_space = clip_length - song_duration
        empty_space_one_side = (empty_space / 2)
        empty_offset = song_start - empty_space_one_side

        # Read in file
        audio_array, sampling_rate = lr.load(this_filename, 
                                             duration=10,
                                             offset = empty_offset)

        # Use this to see when the spectrogram is being extracted.
        # print(empty_offset, song_start, song_end, song_end + empty_space_one_side)

        # Create the Fourier Transform 
        # (which isolates the amplitude level at each frequency)
        X_spec = librosa.stft(audio_array,
                              n_fft = 1024)

        # Converted to Decibels
        spec_db = librosa.core.amplitude_to_db(X_spec)

        if show_specs:
            # Display the spectrogram
            ax = librosa.display.specshow(spec_db)
            plt.colorbar()
            plt.show()

        ax = librosa.display.specshow(spec_db)
        
        pic_filename = str(this_row.recording_id) + "_idx_" + str(i) + "_.png"
        new_save_filename = save_dir + pic_filename
        
        # print(new_save_filename)
        
        plt.savefig(new_save_filename, pad_inches = 0, bbox_inches = 'tight')

        plt.close("all")
        librosa.cache.clear()

    except:
        try:
            skipped.append(base_filename)
        except:
            pass
    
print("skipped: ", skipped)